In [1]:
import os
import numpy as np
import pandas as pd
import ashrae_scripts as scp
import gc

dir = !ls -a
dir = !dir /b
project_folder = 'ashrae-energy-prediction/'
output_compression_type = 'gzip'
if ('kernel-metadata.json' in dir):
    # Local environment
    src = 'Local'
    compression_type = 'gzip'
    data_folder = '../../../data/'
    output_folder = '../../../data/'
    df_train_path = data_folder + project_folder + 'train.csv.gz'
    df_bldg_meta_path = data_folder + project_folder + 'building_metadata.csv.gz'
    df_weather_train_path = data_folder + project_folder + 'weather_train.csv.gz'  
else:
    # Kaggle environment
    src = 'Kaggle'
    compression_type = None
    data_folder = '../input/'
    output_folder = ''
    df_train_path = data_folder + project_folder + 'train.csv'
    df_bldg_meta_path = data_folder + project_folder + 'building_metadata.csv'
    df_weather_train_path = data_folder + project_folder + 'weather_train.csv'
        
print('Environment set to [{env}]'.format(env=src))

Environment set to [Kaggle]


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ashrae-energy-prediction/weather_train.csv
/kaggle/input/ashrae-energy-prediction/test.csv
/kaggle/input/ashrae-energy-prediction/weather_test.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/working/__notebook__.ipynb
/kaggle/working/__output__.json


In [3]:
debug = False
if debug:
    rows = 1000
else:
    rows = None

In [4]:
init_dtype_df = {'building_id': 'int16'
              , 'meter': 'category'
              , 'timestamp': 'str'
              , 'meter_reading': 'float32'}
df_train = pd.read_csv(df_train_path
                       , dtype=init_dtype_df
                       , nrows=rows
                       , compression=compression_type)

In [5]:
init_dtype_bldg = {'site_id': 'str'
              , 'building_id': 'int16'
              , 'primary_use': 'category'
              , 'square_feet': 'float32'
              , 'floor_count': 'float16'
              , 'year_built': 'float16'}
df_bldg_meta = pd.read_csv(df_bldg_meta_path
                           , dtype=init_dtype_bldg
                           , compression=compression_type)

In [6]:
init_dtype_weather = {'timestamp': 'str'
              , 'site_id': 'str'
              , 'precip_depth_1_hr': 'float16'
              , 'wind_direction': 'float16'
              , 'wind_speed': 'float16'
              , 'sea_level_pressure': 'float32'
              , 'dew_temperature': 'float32'
              , 'air_temperature': 'float32'}
df_weather_train = pd.read_csv(df_weather_train_path
                               , dtype=init_dtype_weather
                               , compression=compression_type)

In [7]:
df_bldg_meta = scp.get_bldg_meta(df_bldg_meta)

In [8]:
df_weather_train = scp.get_weather(df_weather_train)

In [9]:
df_train = scp.process_df(df_train, df_bldg_meta, df_weather_train)

In [10]:
df_train.to_csv(output_folder + 'df_train_filled.csv.gz'
                , index=False
                , compression=output_compression_type)

In [11]:
del df_train
del df_weather_train
gc.collect()

22

In [12]:
#np.finfo(np.float16).max